# CÁLCULO DE VIGA PROTENDIDA - VP14

**OBRA:** Carnetti Open Mall

**Cliente:** JD Construções e Pré-Fabricados

**Data:** 14/03/2022

**Cálculo:** Gabriel Garcia

<img src="img/logoRS.jpg" alt="RS Engenharia & Arquitetura" width="200px"/>

Todas as grandezas apresentadas possuem como unidades base kN [força] e cm [comprimento].

In [1]:
import handcalcs.render
import pandas as pd
from math import floor,ceil,sqrt,exp
from numpy import log
from ggcp import calcpsi, linha_neutra_cp_ret
    
pi = 3.141592653

# Informações Iniciais:

## Definição do EPS:

In [2]:
%%render

h_eps = 30
e_eps = 12

<IPython.core.display.Latex object>

## Geometria da Viga:

In [3]:
%%render
# Geometria da Viga:

h = 70
b = 30
Area = b*h - e_eps*h_eps
Inercia = b*h**3/12 - e_eps*(h_eps**3)/12
W_i = Inercia/(h/2)
W_s = Inercia/(h/2)
Vao = 1630

## Número de seções para dividir o vão:
# Escolher número impar
nS = 11

<IPython.core.display.Latex object>

## Propriedades dos Materiais:

In [4]:
%%render
# Propriedades dos Materiais:

f_ck = 4
f_ctm = 0.03*(10*f_ck)**(2/3)
f_ptk = 190
f_pyd = 148.7
f_yk = 50
f_cj = 3.2

E_s = 21000
E_p = 20000

<IPython.core.display.Latex object>

In [5]:
%%render 2 short

f_c = f_ck*0.85/1.4
E_ci = 560*(f_ck*10)**(1/2)
E_cs = E_ci*(0.8+0.2*f_ck/8)

E_cit0 = (f_cj/f_ck)**(1/2)*E_ci
E_cst0 = E_cit0*(0.8+0.2*f_cj/8)

f_yd = f_yk/1.15
sigma_p0 = 0.8*f_ptk

<IPython.core.display.Latex object>

## Carregamentos:

In [6]:
%%render 2 short

Q_t0 = 0.000025*Area
Q_tF = Q_t0 + (50+50+50)*6/100/100


<IPython.core.display.Latex object>

## Esforços no Centro do Vão:

In [7]:
%%render 2 short
M_F = (Q_tF)*Vao**2*0.125
M_0 = (Q_t0)*Vao**2*0.125
V_F = (Q_tF)*Vao/2

<IPython.core.display.Latex object>

# Deteminação da Força de Protensão:

## Limites de Tensão:

In [8]:
%%render short

sigma_ct0 = -1.2*0.03*(f_cj*10)**(2/3)
sigma_cc0 = 0.7*f_cj
sigma_ctF = -1.2*0.03*(f_ck*10)**(2/3)
sigma_ccF = 0.5*f_ck

r_0 = 0.95
r_F = 0.80

<IPython.core.display.Latex object>

## Cálculo da Força de Protensão:

In [22]:
%%render short 2

e_p = 25.8

P_1 = (sigma_cc0+M_0/W_i)/(r_0*(1/Area + e_p/W_i)) 
P_2 = (sigma_ct0-M_0/W_s)/(r_0*(1/Area - e_p/W_s)) 
P_3 = (sigma_ctF+M_F/W_i)/(r_F*(1/Area + e_p/W_i))
P_4 = (sigma_ccF-M_F/W_s)/(r_F*(1/Area - e_p/W_s))

##Adotar valores entre:
P_min = max(P_3, P_4)
P_max = min(P_1, P_2)


<IPython.core.display.Latex object>

## Escolha da Protensão:

In [23]:
%%render 2
# Força de Protensão Adotada:

P = P_max
phi_1 = 1.52
A_p1 = 1.43

## Número de cordoalhas para adotar:
n = floor(P/(A_p1*sigma_p0))

<IPython.core.display.Latex object>

## Ancoragem das Cordoalhas:

### Comprimento de Ancoragem Básico:

In [24]:
%%render short
f_pyd = 0.9* f_ptk / 1.15

eta_p1 = 1.2

# Valores de Eta_p1:
#     
#     1.0 para fios lisos
#     1.2 para cordoalhas de 3 e 7 fios
#     1.5 para fios dentados

eta_p2 = 1.0

# Valores de Eta_p2:
#     
#     1.0 para situação de boa aderência
#     0.7 para situação de má aderência

f_ctd = (0.7*0.03*(f_cj*10)**(2/3))/1.4
f_bpd = eta_p1 * eta_p2 * f_ctd

l_bp = (7*phi_1)/36 * f_pyd/f_bpd

<IPython.core.display.Latex object>

### Cálculo do comprimento de Transferência:

In [25]:
%%render short

## para fios lisos ou dentados:
l_bpt1 = 0.7 * l_bp * sigma_p0/f_pyd

## para cordoalhas de 3 e 7 fios:
l_bpt2 = 0.5 * l_bp * sigma_p0/f_pyd

## para transferência não gradual, os valores devem ser multiplicados por 1.25
## valor adotado:

l_bpt = 1.0 * l_bpt2

l_p = (h**2 + (0.6*l_bpt)**2)**(0.5)


<IPython.core.display.Latex object>

#  Esforços nas Seções:

## Momento Fletor e Cortante:

In [26]:
esforcos = []
esforcos_t0 = []
for i in range(3):
    esforcos.append( [0]*(nS) )
    esforcos_t0.append( [0]*(nS) )
    
# primeiro vetor contem X (cm)
# segundo vetor contem Momentos (kNcm)
# terceiro vetor contém cortante (kN)

for i in range(nS):
    esforcos[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos[1][i] = round( 0.5*Q_tF*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos[2][i] = round( 0.5*Q_tF*(Vao-2*esforcos[0][i]), 1 )
    esforcos_t0[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos_t0[1][i] = round( 0.5*Q_t0*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos_t0[2][i] = round( 0.5*Q_t0*(Vao-2*esforcos[0][i]), 1 )
    
    
tabela = pd.DataFrame(esforcos)
tabela = tabela.transpose()
tabela.rename(columns={0:'Pos (cm)',1:'M (kNcm)',2:'V (kN)'},inplace = True)
display(tabela)

,Pos (cm),M (kNcm),V (kN)
0,0.0,0.0,108.8
1,163.0,15961.3,87.0
2,326.0,28375.7,65.3
3,489.0,37243.1,43.5
4,652.0,42563.5,21.8
5,815.0,44337.0,0.0
6,978.0,42563.5,-21.8
7,1141.0,37243.1,-43.5
8,1304.0,28375.7,-65.3
9,1467.0,15961.3,-87.0


# Escalonamento da Protensão:

## Limites de Tensão na Seção S1:

São testados os limites de tensão para t0:

Limitação pela borda superior:

In [27]:
%%render short

M_s1 = round( 0.5*Q_t0*esforcos[0][1]*(Vao-esforcos[0][1]) ,1) # Momento de peso próprio na seção s1
P_1 = (sigma_cc0+M_s1/W_i)/(r_0*(1/Area + e_p/W_i))
P_2 = (sigma_ct0-M_s1/W_s)/(r_0*(1/Area - e_p/W_s))


<IPython.core.display.Latex object>

In [28]:
%%render 1

P_s1 = min(P_1 , P_2)

n_s1 = floor(P_s1/(A_p1*sigma_p0))


<IPython.core.display.Latex object>

## Escolha do Número de Cordoalhas:

In [42]:
%%render

# Vetor com o número de cordoalhas em cada ponto de escalonamento:

qntCordoalhas = [5,7]

# Vetor com a excentricidade da protensão em cada ponto de escalonamento:

e_p_i = [25.8,25.7]

<IPython.core.display.Latex object>

# Esforços Devidos À Protensão:

In [43]:
A_p = []

for i in range(len(qntCordoalhas)):
    A_p.append(qntCordoalhas[i] * A_p1)

W_cp = []

for i in range(len(qntCordoalhas)):
    W_cp.append(round(Inercia/e_p_i[i],0))
    
alpha_p0 = E_p / E_cst0

K_p0 = []
for i in range(len(qntCordoalhas)):
    K_p0.append(round(alpha_p0*A_p[i]*(1/Area+e_p_i[i]/W_cp[i]),4))
    
sigma_p = []
for i in range(len(qntCordoalhas)):
    sigma_p.append(round(sigma_p0/(1+K_p0[i]),2))
    
N_p = []
for i in range(len(qntCordoalhas)):
    N_p.append(round(-sigma_p[i]*A_p[i],1))
    
M_p = []
for i in range(len(qntCordoalhas)):
    M_p.append(round(N_p[i]*e_p_i[i],1))
    
tabela1 = pd.DataFrame([A_p, W_cp, K_p0, sigma_p, N_p, M_p])
tabela1 = tabela1.transpose()
tabela1.rename(columns={0:'Ap',1:'Wcp',2:'Kp0',3:'Sigma,p',4:'Np',5:'Mp'},inplace = True)
display(tabela1)

,Ap,Wcp,Kp0,"Sigma,p",Np,Mp
0,7.15,32190.0,0.0706,141.98,-1015.2,-26192.2
1,10.01,32315.0,0.0984,138.38,-1385.2,-35599.6


# Escalonamento da Força de Protensão:

In [44]:
M_g = []
x1 = []
x2 = []
x1_anc = []
x2_anc = []

for i in range(len(qntCordoalhas)):
    M_g.append(round( W_s*( (N_p[i]/Area) -(M_p[i]/W_s) + sigma_ct0),2) )
    #print("M_g["+str(i)+"]:",M_g[i])
        
for i in range(len(qntCordoalhas)):
    # O erro acontece no caso da raiz de delta retornar um número complexo
    try:
        c = 2*M_g[i]/Q_t0
        delta = (-Vao)**2 - 4*c
        #print("Delta "+str(i)+":",round(delta,2))
        x1.append(round( (Vao-delta**0.5)/2  ,2))
        x2.append(round( (Vao+delta**0.5)/2  ,2))
    except:
        print("##   ATENÇÃO!   ##")
        raise Exception("ERRO: ESCALONAR MELHOR A PROTENSÃO!")    

for i in range(len(qntCordoalhas)):
    x1_anc.append(x1[i]-round(l_bpt,2))
    x2_anc.append(x2[i]+round(l_bpt,2))
    if x1_anc[i] < 0:
        x1_anc[i] = 0
    if x2_anc[i] > Vao:
        x2_anc[i] = Vao
    
tabela2 = pd.DataFrame([M_g, x1,x2, x1_anc,x2_anc])
tabela2 = tabela2.transpose()
tabela2.rename(columns={0:'Mg',1:'x1',2:'x2',3:'x1-lbpt',4:'x2+lbpt'},inplace = True)
display(tabela2)

,Mg,x1,x2,x1-lbpt,x2+lbpt
0,3737.72,113.31,1516.69,0.00,1630.00
1,8099.39,274.78,1355.22,150.97,1479.03


# Perdas Progressivas da Força de Protensão:

## Fluência do Concreto:

### Fluência Rápida Irreversível

In [45]:
%%render params

# VER ITEM 12.3.3 da NBR6118:2014
# Valores para adotar:
# s = 0.38 para concreto de cimento CP-III e CP-IV
# s = 0.25 para concreto de cimento CP-I e CP-II
# s = 0.20 para concreto de cimento CP-V
s = 0.20
t_0 = 10
t_F = 10000


beta_0 = exp(s*(1-(28/t_0)**0.5))
beta_F = exp(s*(1-(28/t_F)**0.5))

phi_a = 0.8*(1 - (beta_0/beta_F))

<IPython.core.display.Latex object>

In [46]:
if s == 0.38:
    print("Adotado s = 0.38 para concreto de cimento CP-III ou CP-IV")
elif s == 0.25:
    print("Adotado s = 0.25 para concreto de cimento CP-I ou CP-II")
else:
    print("Adotado s = 0.20 para concreto de cimento CP-V (ARI) ")

Adotado s = 0.20 para concreto de cimento CP-V (ARI) 


### Fluência Lenta Irreversível:

In [47]:
%%render params 2

U = 55 #umidade média do ar

phi_c1 = 4.45 - 0.035*U # para abatimentos entre 5cm e 9cm

gamma = 1 + exp(-7.8 + 0.1 * U)

u_ar = ( 2*h + 2*b )

h_fic = gamma*(2*Area)/u_ar

phi_c2 = (42+h_fic)/(20+h_fic)

phi_fF = phi_c1*phi_c2

h_fm = h_fic/100 # espessura fictícia em metros

A_beta_f = 42*h_fm**3 - 350*h_fm**2 + 588*h_fm + 113
B_beta_f = 768*h_fm**3 - 3060*h_fm**2 + 3234*h_fm - 23
C_beta_f = -200*h_fm**3 + 13*h_fm**2 + 1090*h_fm + 183
D_beta_f = 7579*h_fm**3 - 31916*h_fm**2 + 35343*h_fm + 1931

beta_f_t0 = (t_0**2 + A_beta_f*t_0 + B_beta_f )/(t_0**2 + C_beta_f*t_0 + D_beta_f)
beta_f_tF = (t_F**2 + A_beta_f*t_F + B_beta_f )/(t_F**2 + C_beta_f*t_F + D_beta_f) 

<IPython.core.display.Latex object>

### Fluência Lenta Reversível:

In [48]:
%%render params

phi_dF = 0.4

beta_d = (t_F-t_0+20)/(t_F-t_0+70)

<IPython.core.display.Latex object>

### Fluência Total:

In [49]:
%%render 2 params
phi_t = phi_a + phi_fF*(beta_f_tF - beta_f_t0) + phi_dF * beta_d

<IPython.core.display.Latex object>

### Retração do Concreto:

In [50]:
%%render 2 params

epsilon_1s = (-8.09 + U/15 - U**2/2284 + U**3/133765 - U**4/7608150)*10**-4

epsilon_2s = (33+2*h_fic)/(20.8 + 3*h_fic)

A_beta_s = 40  
B_beta_s = 116*h_fm**3 - 282*h_fm**2 + 220*h_fm - 4.8  
C_beta_s = 2.5*h_fm**3 - 8.8*h_fm + 40.7  
D_beta_s = -75*h_fm**3 + 585*h_fm**2 + 496*h_fm - 6.8  
E_beta_s = -169*h_fm**4 + 88*h_fm**3 + 584*h_fm**2 - 39*h_fm + 0.8  

beta_s_t0 = ((t_0/100)**3 + A_beta_s*(t_0/100)**2 +  B_beta_s*(t_0/100))/((t_0/100)**3 + C_beta_s*(t_0/100)**2 +  D_beta_s*(t_0/100) + E_beta_s)

beta_s_tF = ((t_F/100)**3 + A_beta_s*(t_F/100)**2 +  B_beta_s*(t_F/100))/((t_F/100)**3 + C_beta_s*(t_F/100)**2 +  D_beta_s*(t_F/100) + E_beta_s)

epsilon_cs = epsilon_1s*epsilon_2s*(beta_s_tF - beta_s_t0)


<IPython.core.display.Latex object>

## Relaxação do Aço de Protensão:

In [51]:
relax = []
for i in range(7):
    relax.append([0]*nS)

for i in range (nS):
    relax[0][i] = int(i+1)
    relax[1][i] = esforcos[0][i] #posição x da seção
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            relax[2][i] = round(sigma_p[j],2)
            break
    relax[3][i] = round(relax[2][i]/f_ptk,3)
    relax[4][i] = round(calcpsi(relax[3][i]),3)
    relax[5][i] = round(relax[4][i]*2.5,3)
    relax[6][i] = round(-log(1-relax[5][i]),3)
    
    
tabela3 = pd.DataFrame(relax)
tabela3 = tabela3.transpose()
tabela3.rename(columns={0:'Seção',1:'Pos',2:'Sigma,p',3:'Sigma,p/fptk',4:'\u03A81000',5:'\u03A8,inf',6:'X,inf'},inplace = True)
display(tabela3)

,Seção,Pos,"Sigma,p","Sigma,p/fptk",Ψ1000,"Ψ,inf","X,inf"
0,1.0,0.0,141.98,0.747,0.030,0.075,0.078
1,2.0,163.0,138.38,0.728,0.028,0.070,0.073
2,3.0,326.0,138.38,0.728,0.028,0.070,0.073
3,4.0,489.0,138.38,0.728,0.028,0.070,0.073
4,5.0,652.0,138.38,0.728,0.028,0.070,0.073
5,6.0,815.0,138.38,0.728,0.028,0.070,0.073
6,7.0,978.0,138.38,0.728,0.028,0.070,0.073
7,8.0,1141.0,138.38,0.728,0.028,0.070,0.073
8,9.0,1304.0,138.38,0.728,0.028,0.070,0.073
9,10.0,1467.0,138.38,0.728,0.028,0.070,0.073


## Cálculo das Perdas Progressivas:

In [52]:
progressivas = []

for i in range(4):
    progressivas.append([0] * nS)
    
alpha_p = E_p/E_cs
    
for i in range(nS):
    pos_i = relax[1][i]
    momento = 0.5*Q_t0*pos_i*(Vao-pos_i)
    #momento = esforcos[1][i]
    #print("Momento seção "+str(i)+": "+str(int(momento)))
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    progressivas[0][i] = round(Npi/Area + Mpi/Wcpi + momento/Wcpi,2)
    #print(Npi,Mpi)
    K_p = alpha_p*A_pp*(1/Area + e_pp/Wcpi)
    progressivas[1][i] = round( (( progressivas[0][i]*phi_t/E_cs + epsilon_cs - sigma_pi/E_p*relax[6][i] ) / (1 + relax[6][i] + K_p*(1+0.5*phi_t)) )*E_p,2)
    progressivas[2][i] = -round(progressivas[1][i]*A_pp  ,2)
    progressivas[3][i] = round(progressivas[2][i]*e_pp ,1)
    
    

tabela4 = pd.DataFrame(progressivas)
tabela4 = tabela4.transpose()
tabela4.rename(columns={0:'Sigma,c,t0',1:'\u0394Sigma,p',2:'\u0394Np',3:'\u0394Mp'},inplace = True)
display(tabela4)

,"Sigma,c,t0","ΔSigma,p",ΔNp,ΔMp
0,-1.40,-41.28,295.15,7614.9
1,-1.74,-44.37,444.14,11414.4
2,-1.61,-42.15,421.92,10843.3
3,-1.52,-40.61,406.51,10447.3
4,-1.47,-39.76,398.00,10228.6
5,-1.45,-39.42,394.59,10141.0
6,-1.47,-39.76,398.00,10228.6
7,-1.52,-40.61,406.51,10447.3
8,-1.61,-42.15,421.92,10843.3
9,-1.74,-44.37,444.14,11414.4


# Tensões Normais nas Bordas:

## Bordas Superiores:

In [53]:
tensaosup = []

for i in range(3):
    tensaosup.append([0]*nS)
    
for i in range(nS):
    tensaosup[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaosup[1][i] = round( -abs(Npx/Area) + abs(Mpx/W_s) - abs(esforcos[1][i]/W_s) ,3)
    if tensaosup[1][i] > -sigma_ccF:
        tensaosup[2][i] = "OK"
    else:
        tensaosup[2][i] = "--> NAO OK <--"

tabela5 = pd.DataFrame(tensaosup)
tabela5 = tabela5.transpose()
tabela5.rename(columns={0:'Pos (cm)',1:'Sigma,s',2:'OK'},inplace = True)
display(tabela5)

,Pos (cm),"Sigma,s",OK
0,0.0,0.369,OK
1,163.0,-0.194,OK
2,326.0,-0.706,OK
3,489.0,-1.072,OK
4,652.0,-1.292,OK
5,815.0,-1.365,OK
6,978.0,-1.292,OK
7,1141.0,-1.072,OK
8,1304.0,-0.706,OK
9,1467.0,-0.194,OK


## Bordas Inferiores:

In [54]:
tensaoinf = []

for i in range(3):
    tensaoinf.append([0]*nS)
    
for i in range(nS):
    tensaoinf[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaoinf[1][i] = round( -abs(Npx/Area) - abs(Mpx/W_i) + abs(esforcos[1][i]/W_i) ,3)
    if tensaoinf[1][i] < -sigma_ctF:
        tensaoinf[2][i] = "OK"
    else:
        tensaoinf[2][i] = "--> NAO OK <--"
    
tabela6 = pd.DataFrame(tensaoinf)
tabela6 = tabela6.transpose()
tabela6.rename(columns={0:'Pos (cm)',1:'Sigma,i',2:'OK'},inplace = True)
display(tabela6)

,Pos (cm),"Sigma,i",OK
0,0.0,-1.197,OK
1,163.0,-0.887,OK
2,326.0,-0.401,OK
3,489.0,-0.053,OK
4,652.0,0.157,OK
5,815.0,0.226,OK
6,978.0,0.157,OK
7,1141.0,-0.053,OK
8,1304.0,-0.401,OK
9,1467.0,-0.887,OK


# Armaduras Passivas:

## Armadura da Cunha de Tração:

In [55]:
sigma_cs0 = -10000
pos = -1
for i in range (0,nS):
    if sigma_cs0 < tensaosup[1][i]:
        sigma_cs0 = tensaosup[1][i]
        pos = i

sigma_ci0 = tensaoinf[1][pos]

print("Seção crítica de cálculo: seção",pos)

Seção crítica de cálculo: seção 0


In [56]:
%%render 2

x = sigma_cs0/(sigma_cs0 - sigma_ci0) * h

S_x = b*x**2/2

R_ct = sigma_cs0/x*S_x

A_st = R_ct/25.00

phi_st = 1.6

N_st = ceil(A_st/((phi_st/2)**2*pi))

<IPython.core.display.Latex object>

## Pré-Alongamento de Leito:

In [57]:
prealong = []

for i in range(5):
    prealong.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
                sigma_pw = sigma_p[j]
                Npw = N_p[j]
                Mpw = M_p[j]
                Wcpw = W_cp[j]
                break
    prealong[0][i] = Npw + progressivas[2][i]
    prealong[1][i] = Mpw + progressivas[3][i]
    prealong[2][i] = round(prealong[0][i]/Area + prealong[1][i]/Wcpw ,2)
    prealong[3][i] = round( sigma_pw + progressivas[1][i] - alpha_p*prealong[2][i] ,2)
    prealong[4][i] = round(prealong[3][i]/E_p,5)

tabela7 = pd.DataFrame(prealong)
tabela7 = tabela7.transpose()
tabela7.rename(columns={0:'Np,F',1:'Mp,F',2:'Sigma,cpF', 3:'Sigma,PN',4:'e,pn'},inplace = True)
display(tabela7)

,"Np,F","Mp,F","Sigma,cpF","Sigma,PN","e,pn"
0,-720.05,-18577.3,-0.99,106.91,0.00535
1,-941.06,-24185.2,-1.29,102.10,0.00510
2,-963.28,-24756.3,-1.32,104.51,0.00523
3,-978.69,-25152.3,-1.34,106.18,0.00531
4,-987.20,-25371.0,-1.35,107.09,0.00535
5,-990.61,-25458.6,-1.36,107.49,0.00537
6,-987.20,-25371.0,-1.35,107.09,0.00535
7,-978.69,-25152.3,-1.34,106.18,0.00531
8,-963.28,-24756.3,-1.32,104.51,0.00523
9,-941.06,-24185.2,-1.29,102.10,0.00510


## Armadura Passiva de Flexão:

![formulario](img/formulario.png)

In [58]:
d_p = [] # braço de alavanca da armadura ativa
d_s = [] # braço de alavanca da armadura passiva

for i in range(len(qntCordoalhas)):
    d_p.append(e_p_i[i] + h/2)
    d_s.append(h-5) # estimativa para ds
    
A_s = []
for i in range(8):
    A_s.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pm = sigma_p[j]
            Npm = N_p[j]
            Mpm = M_p[j]
            Wcpm = W_cp[j]
            A_pm = A_p[j]
            d_sm = d_s[j]
            d_pm = d_p[j]
            break
    Md = esforcos[1][i]*1.4
    retorno = linha_neutra_cp_ret(b,h,f_c,A_pm,prealong[4][i],d_pm,d_sm,Md)
    A_s[0][i] = int(Md)
    A_s[1][i] = retorno[0]
    A_s[2][i] = round(retorno[0]/d_sm,3)
    A_s[3][i] = retorno[2]
    A_s[5][i] = retorno[1]
    A_s[4][i] = retorno[4]
    A_s[6][i] = retorno[5]
    A_s[7][i] = round((0.8*b*f_c*retorno[0]-retorno[3])/retorno[4],2)
    

tabela8 = pd.DataFrame(A_s)
tabela8 = tabela8.transpose()
tabela8.rename(columns={0:'Md', 1:'x',2:'x/d',3:'eps,s',4:'sigma,s',5:'eps,p',6:'sigma,p',7:'As'},inplace = True)
display(tabela8)

,Md,x,x/d,"eps,s","sigma,s","eps,p","sigma,p",As
0,0.0,1.19,0.018,0.18810,43.48,0.18054,148.7,-22.86
1,22345.0,7.98,0.123,0.02501,43.48,0.02771,148.7,-23.54
2,39725.0,13.26,0.204,0.01366,43.48,0.01723,148.7,-16.46
3,52140.0,17.29,0.266,0.00966,43.48,0.01356,148.7,-11.06
4,59588.0,19.84,0.305,0.00797,43.48,0.01202,148.7,-7.64
5,62071.0,20.71,0.319,0.00748,43.48,0.01159,148.7,-6.47
6,59588.0,19.84,0.305,0.00797,43.48,0.01202,148.7,-7.64
7,52140.0,17.29,0.266,0.00966,43.48,0.01356,148.7,-11.06
8,39725.0,13.26,0.204,0.01366,43.48,0.01723,148.7,-16.46
9,22345.0,7.98,0.123,0.02501,43.48,0.02771,148.7,-23.54


# Dimensionamento ao Esforço Cortante:

## Verificação da Diagonal Comprimida:

In [59]:
%%render

V_d = V_F * 1.4
tau_wd = V_d/((b-e_eps)*d_sm)
tau_wu = 0.27*(1-f_ck/25)*f_ck/1.4


<IPython.core.display.Latex object>

In [60]:
if tau_wu > tau_wd:
    print("OK!")
else:
    print("NÃO OK!")

OK!


## Cálculo da Armadura de Corte:

### Armadura Mínima:

In [61]:
%%render short

rho_min = 0.2*f_ctm/f_yk
A_sws = rho_min*b

s = 15

A_sw_min = A_sws*s

nr = 2

phi_min = 2*(A_sw_min/(pi*nr))**0.5


<IPython.core.display.Latex object>

### Armadura Calculada:

In [62]:
i = int((nS+1)/2)
M_pf = prealong[1][i]
N_pf = prealong[0][i]

In [63]:
%%render

sigma_ciF = N_pf/Area + M_pf/W_i
M_0 = -sigma_ciF*W_i
M_d_max = 1.4*M_F

V_c0 = 0.6*f_ctd*b*d_sm
V_c = (1+M_0/M_d_max)*V_c0

V_sw = 1.4*V_F - V_c

A_sws = V_sw/(0.9*d_sm*f_yd)

s = 20
nr = 2
phi_min = 2*(A_sws/(pi*nr))**0.5

<IPython.core.display.Latex object>

## Armadura de Introdução da Força de Protensão:

In [64]:
N_p1 = -N_p[0]

In [65]:
%%render

A_swi = (0.25*N_p1)/25.00

<IPython.core.display.Latex object>

Esta armadura deve ser distribuída num trecho equivalente à 70% do comprimento de regularização.

In [66]:
%%render

l_d = l_p*0.7

<IPython.core.display.Latex object>

# Cálculo da Flecha Elástica:

**Flecha devida à protensão:**

Ap = área de aço de protensão

b = ponto de início da atuação da força

Delta = flecha causada por cada conjunto de cordoalhas

In [67]:

delta_p = []

delta_pF = 0

for i in range(len(qntCordoalhas)):
    delta_p.append([0] * 3)
    delta_p[i][0] = A_p[i]
    delta_p[i][1] = x1_anc[i]
    if(delta_p[i][1] == 0):
        delta_p[i][2] = round(M_p[i]*Vao**2/(8*E_cs*Inercia),2)
    else:
        delta_p[i][1] = (x1_anc[i]+x1[i])/2
        delta_p[i][2] = round( (M_p[i]*(Vao**2-3*delta_p[i][1]**2-(Vao/2)**2)*(Vao/2))
                             /(6*E_cs*Inercia*Vao) ,2)
        
for i in range(len(qntCordoalhas)):
    delta_pF += delta_p[i][2]
    
tabela9 = pd.DataFrame(delta_p)
tabela9.rename(columns={0:'Ap',1:'b',2:'Delta'},inplace = True)

display(tabela9)

if (delta_p[0][0]) < 0:
    raise Exception("ERRO")

,Ap,b,Delta
0,7.15,0.000,-3.29
1,10.01,212.875,-2.08


In [68]:
%%render
delta_pF

limite = Vao/250

delta_i = 5/384 * (Q_tF*Vao**4)/(Inercia*E_cs)

delta_t0 = delta_pF + delta_i

delta_tF = 2.5*delta_t0

<IPython.core.display.Latex object>